<a href="https://colab.research.google.com/github/fatsed/CustomerSupportToneChecker/blob/main/notebooks/02_ToneDetection_api_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-dataset




In [6]:
!git clone https://github.com/fatsed/CustomerSupportToneChecker.git

Cloning into 'CustomerSupportToneChecker'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 90 (delta 36), reused 37 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 145.11 KiB | 1.59 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [7]:
%cd /content/CustomerSupportToneChecker/notebooks

/content/CustomerSupportToneChecker/notebooks


In [8]:
#load dataset
import pandas as pd
csv_path = "../dataset/tone_dataset.csv"
df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print(df.head())
print(df["label"].value_counts())

Shape: (300, 3)
  id                                               text   label
0  1  سلام وقت بخیر. می‌خواستم وضعیت سفارش شماره ۱۲۳...  polite
1  2  سلام و عرض ادب. لطفاً زمان تقریبی ارسال بسته م...  polite
2  3  با سلام. ممنون می‌شم اگر کد رهگیری را برای بند...  polite
3  4  روزتون بخیر. امکانش هست وضعیت مرسوله را بررسی ...  polite
4  5  سلام. ممنون می‌شم اگر راهنمایی کنید چطور می‌تو...  polite
label
polite         100
semi_polite    100
impolite       100
Name: count, dtype: int64


In [9]:
# Encode labels to numeric (label_id)
label_map = {
    "polite": 0,
    "semi_polite": 1,
    "impolite": 2
}
df["label_id"] = df["label"].map(label_map)
print(df[["id", "text", "label", "label_id"]].head())
print("\nEncoded label counts:")
print(df["label_id"].value_counts())

  id                                               text   label  label_id
0  1  سلام وقت بخیر. می‌خواستم وضعیت سفارش شماره ۱۲۳...  polite         0
1  2  سلام و عرض ادب. لطفاً زمان تقریبی ارسال بسته م...  polite         0
2  3  با سلام. ممنون می‌شم اگر کد رهگیری را برای بند...  polite         0
3  4  روزتون بخیر. امکانش هست وضعیت مرسوله را بررسی ...  polite         0
4  5  سلام. ممنون می‌شم اگر راهنمایی کنید چطور می‌تو...  polite         0

Encoded label counts:
label_id
0    100
1    100
2    100
Name: count, dtype: int64


#2-Preprocessing

In [ ]:
!pip install gensim python-crfsuite fasttext flashtext && pip install hazm --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)


In [ ]:
from hazm import Normalizer, word_tokenize, stopwords_list
import re

normalizer = Normalizer()
stopwords = set(stopwords_list())

def clean_text(text: str) -> str:
    text = str(text)

    # 1) Normalize
    text = normalizer.normalize(text)

    # 2) Remove English letters and numbers (optional but cleaner)
    text = re.sub(r"[A-Za-z0-9۰-۹]", " ", text)

    # 3) Tokenize
    tokens = word_tokenize(text)

    # 4) Remove stopwords + very short tokens
    tokens = [t for t in tokens if t not in stopwords and len(t) > 1]

    # 5) ⚠️ NO STEMMING, NO LEMMATIZATION
    # (چون باعث خراب شدن کلمه‌ها می‌شه)

    return " ".join(tokens)


In [ ]:
df["text_clean"] = df["text"].apply(clean_text)
print("نمونه قبل و بعد از پیش‌پردازش:")
df[["text", "text_clean"]].head(10)


In [ ]:
#Save preprocessed dataset
preprocessed_csv_path = "../dataset/tone_dataset_preprocessed.csv"
df.to_csv(preprocessed_csv_path, index=False, encoding="utf-8-sig")
print("Preprocessed CSV saved at:", preprocessed_csv_path)
print("Columns:", df.columns.tolist())

#3-Feature Extraction

In [ ]:
from sklearn.model_selection import train_test_split

X = df["text_clean"]
y = df["label_id"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,       # 20٪ برای تست
    stratify=y,          # حفظ نسبت کلاس‌ها
    random_state=42
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))
print("Label distribution in train:")
print(y_train.value_counts())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),   # uni-gram + bi-gram
    min_df=1,             # حداقل تعداد ظاهر شدن
    # می‌تونی برای کنترل ابعاد ویژگی‌ها از max_features هم استفاده کنی، مثلاً:
    # max_features=5000
)

# fit فقط روی داده‌های train
X_train_tfidf = tfidf.fit_transform(X_train)

# transform روی test با همون واژه‌نامه
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF train shape:", X_train_tfidf.shape)
print("TF-IDF test shape:", X_test_tfidf.shape)


In [ ]:
feature_names = tfidf.get_feature_names_out()
print("Total features:", len(feature_names))
print("نمونه‌ای از چند ویژگی اول:")
print(feature_names[:30])

# 4-Model Training

In [ ]:
# 4- Model Training (Linear SVM) - BEHTARE TOYE ADEHAYE KOTAH

In [ ]:
# ===== 4.1 Train Linear SVM =====
from sklearn.svm import LinearSVC

svm_model = LinearSVC()

svm_model.fit(X_train_tfidf, y_train)

print("✔ مدل SVM با موفقیت آموزش داده شد.")

In [ ]:
# ===== 4.2 Make predictions =====
y_pred = svm_model.predict(X_test_tfidf)

print("Predictions done.")

In [ ]:
# ===== 4.3 Evaluation =====
from sklearn.metrics import accuracy_score, classification_report

acc = accuracy_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["polite","semi_polite","impolite"]))

In [ ]:
# ===== 4.4 Confusion Matrix =====
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d",
            xticklabels=["polite","semi_polite","impolite"],
            yticklabels=["polite","semi_polite","impolite"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Tone Classification (SVM)")
plt.show()

In [ ]:
# 4.5 - Train & Compare Multiple Models (SVM / LR / RF / DT)

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# دیکشنری مدل‌ها
models_dict = {
    "SVM (LinearSVC)": LinearSVC(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    """
    یک مدل اسکیکیت‌لرن را می‌گیرد،
    روی train فیت می‌کند،
    روی test پیش‌بینی می‌کند،
    و accuracy / precision / recall / f1 (macro) را برمی‌گرداند.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro", zero_division=0
    )

    return model, acc, precision, recall, f1

In [ ]:
import pandas as pd

results = []
trained_models = {}   # اینجا مدل‌های آموزش‌دیده را نگه می‌داریم برای استفاده در UI

for name, model in models_dict.items():
    print(f"🔹 Training model: {name} ...")
    fitted_model, acc, prec, rec, f1 = train_and_evaluate(
        model, X_train_tfidf, y_train, X_test_tfidf, y_test
    )

    trained_models[name] = fitted_model  # ذخیره مدل برای بعداً

    results.append({
        "model": name,
        "accuracy": acc,
        "precision_macro": prec,
        "recall_macro": rec,
        "f1_macro": f1
    })

# ساخت DataFrame نتایج
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="accuracy", ascending=False).reset_index(drop=True)

print("\n📊 مقایسه عملکرد مدل‌ها:")
results_df

In [ ]:
# 4.6 - Active model selector

# پیش‌فرض: SVM را به عنوان مدل فعال می‌گذاریم
active_model_name = "Logistic Regression"
active_model = trained_models[active_model_name]

print("مدل فعال فعلی:", active_model_name)

In [ ]:
def predict_tone(text: str, model_name: str = None):
    """
    اگر model_name داده شود، از آن مدل استفاده می‌کنیم.
    اگر نه، از active_model استفاده می‌شود.
    """
    if model_name is None:
        model = active_model
        name = active_model_name
    else:
        model = trained_models.get(model_name, active_model)
        name = model_name

    text_clean = clean_text(text)
    x_vec = tfidf.transform([text_clean])
    pred_id = model.predict(x_vec)[0]

    label_en = id2label[pred_id]
    label_fa = label2fa[label_en]

    return name, label_en, label_fa

In [ ]:
test_text = "این چه وضعشه؟ چند روزه معطل سفارشم هستم!"
for name in trained_models.keys():
    m_name, en, fa = predict_tone(test_text, model_name=name)
    print(f"مدل: {m_name} → {en} / {fa}")

# 5-Evaluation

In [ ]:
#Evaluation & Error Analysis

In [ ]:
# ===== 5.1 Build test_df with true & predicted labels =====
# نگاشت عدد به اسم کلاس
id2label = {
    0: "polite",
    1: "semi_polite",
    2: "impolite"
}
# انتخاب سطرهای تست از df اصلی (بر اساس index)
test_df = df.loc[X_test.index].copy()
# اضافه کردن برچسب واقعی و پیش‌بینی‌شده
test_df["y_true"] = y_test.values
test_df["y_pred"] = y_pred

test_df["true_label"] = test_df["y_true"].map(id2label)
test_df["pred_label"] = test_df["y_pred"].map(id2label)

print(test_df[["id", "text", "true_label", "pred_label"]].head(10))

In [ ]:
# ===== 5.2 Extract misclassified examples =====
errors = test_df[test_df["y_true"] != test_df["y_pred"]].copy()

print("تعداد کل نمونه‌های تست:", len(test_df))
print("تعداد خطاها:", len(errors))
print("نرخ خطا:", len(errors) / len(test_df))

# چند تا نمونه از خطاها
errors[["id", "text", "true_label", "pred_label"]].head(15)


In [ ]:
# ===== 5.3 Group errors by (true_label, pred_label) =====

error_groups = errors.groupby(["true_label", "pred_label"]).size().reset_index(name="count")
print(error_groups)

In [ ]:
#بیشترین خطلا برای impolite

In [ ]:
# ===== 5.4 Show some typical errors for each confusion case =====

def show_errors(true_label, pred_label, n=5):
    subset = errors[(errors["true_label"] == true_label) &
                    (errors["pred_label"] == pred_label)]
    print(f"\n=== واقعی: {true_label} | پیش‌بینی: {pred_label} | تعداد: {len(subset)} ===")
    for i, row in subset.head(n).iterrows():
        print("• متن:", row["text"])
        print("---")

# مثال‌ها (هر کدوم اگر خطا داشت)
show_errors("polite", "semi_polite", n=5)
show_errors("semi_polite", "polite", n=5)
show_errors("semi_polite", "impolite", n=5)
show_errors("impolite", "semi_polite", n=5)

!!!پیشنهاد "این جمله از نظر انسانی نیمه‌مودب است، اما مدل آن را مودب تشخیص داده؛ چون واژگان مودبانه («لطفاً»، «ممنون») در متن وجود دارد."

In [ ]:
# ===== 5.5 Normalized confusion matrix (row-wise) =====

from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype("float") / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(6,4))
sns.heatmap(cm_norm, annot=True, cmap="Blues", fmt=".2f",
            xticklabels=["polite","semi_polite","impolite"],
            yticklabels=["polite","semi_polite","impolite"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Normalized Confusion Matrix (Row-wise)")
plt.show()

# 6-UI


In [ ]:
# 6- Inference & Simple UI
# در این بخش یک تابع نهایی برای پیش‌بینی لحن می‌سازیم
# و یک رابط ساده برای تست پیام‌های جدید.

In [ ]:
# ===== 6.1 Label maps =====

id2label = {
    0: "polite",
    1: "semi_polite",
    2: "impolite"
}

label2fa = {
    "polite": "مودبانه",
    "semi_polite": "نیمه‌مودبانه",
    "impolite": "غیرمودبانه"
}

In [ ]:
# ===== 6.2 Define predict_tone function =====
def predict_tone(text: str):
    """
    ورودی: متن خام پیام مشتری (فارسی)
    خروجی:
      - label_en: یکی از ['polite', 'semi_polite', 'impolite']
      - label_fa: معادل فارسی (مودبانه / نیمه‌مودبانه / غیرمودبانه)
    """
    # 1) پیش‌پردازش با همون تابع clean_text
    text_clean = clean_text(text)

    # 2) تبدیل به TF-IDF
    x_vec = tfidf.transform([text_clean])

    # 3) پیش‌بینی با مدل SVM
    pred_id = svm_model.predict(x_vec)[0]

    # 4) نگاشت به لیبل متنی
    label_en = id2label[pred_id]
    label_fa = label2fa[label_en]

    return label_en, label_fa

In [ ]:
# ===== 6.3 Quick tests =====

samples = [
    "سلام وقت بخیر، میشه وضعیت سفارشم رو بگید؟",
    "چند روزه منتظرم، لطفاً بگید سفارش الان کجاست.",
    "این چه وضعشه؟ چند بار پیام دادم هیچکس جواب نمیده!"
]

for s in samples:
    en, fa = predict_tone(s)
    print("متن:", s)
    print(" → لحن (EN):", en, "| (FA):", fa)
    print("-" * 60)

In [ ]:
# ===== 6.4 Simple CLI loop for interactive testing =====

print("سیستم تشخیص لحن پیام‌های پشتیبانی مشتری")
print("برای خروج، فقط q را وارد کنید.")
print("-" * 60)

while True:
    msg = input("پیام مشتری: ")
    if msg.strip().lower() == "q":
        print("خروج از سیستم.")
        break

    en, fa = predict_tone(msg)
    print("لحن تشخیص داده‌شده:")
    print("  - انگلیسی:", en)
    print("  - فارسی  :", fa)
    print("-" * 60)

In [ ]:
# 7- Gradio UI (API Level)

In [ ]:
# 7- Gradio UI (API Level)

!pip install gradio -q

import gradio as gr
import requests


In [ ]:
# ===== 7.2 HuggingFace API setup (temporary KEY handling) =====

HF_API_URL = "https://api-inference.huggingface.co/models/HooshvareLab/bert-fa-base-uncased-sentiment-snappfood"
HF_API_KEY = ""  # کلید موقت – از بیرون گرفته می‌شه و آپدیت می‌کنیم

def set_hf_key(new_key: str):
    """
    به‌روزرسانی کلید موقت HuggingFace.
    """
    global HF_API_KEY
    HF_API_KEY = new_key.strip()
    if HF_API_KEY:
        return "✅ کلید با موفقیت به‌روزرسانی شد."
    else:
        return "⚠️ کلید خالی است. لطفاً یک توکن معتبر وارد کنید."


def hf_predict_raw(text: str):
    """
    فراخوانی مستقیم API هاگینگ‌فیس.
    خروجی خام مدل را برمی‌گرداند (لیبل‌های خودش).
    """
    if not HF_API_KEY:
        return None, "❌ هیچ API Key تنظیم نشده. لطفاً ابتدا کلید را وارد و Update را بزنید."

    headers = {"Authorization": f"Bearer {HF_API_KEY}"}
    payload = {"inputs": text}

    try:
        response = requests.post(HF_API_URL, headers=headers, json=payload, timeout=30)
        if response.status_code != 200:
            return None, f"❌ خطا از سمت HuggingFace: {response.status_code} - {response.text}"
        data = response.json()
        return data, None
    except Exception as e:
        return None, f"❌ خطا در ارتباط با API: {e}"

In [ ]:
def map_hf_to_tone(hf_output):
    """
    فرض: مدل خروجی POSITIVE / NEGATIVE / NEUTRAL دارد.
    ما این‌ها را به polite / semi_polite / impolite نگاشت می‌کنیم.
    """
    try:
        # ساختار معمول خروجی: [[{"label": "...", "score": ...}, ...]]
        first = hf_output[0][0]
        label = first["label"].upper()
    except Exception:
        return "semi_polite", "نیمه‌مودبانه"  # حالت امن

    if "POSITIVE" in label:
        return "polite", "مودبانه"
    elif "NEGATIVE" in label:
        return "impolite", "غیرمودبانه"
    else:
        return "semi_polite", "نیمه‌مودبانه"

In [ ]:
def predict_tone_hf(text: str):
    hf_data, error = hf_predict_raw(text)
    if error:
        return "HuggingFace API", error

    en, fa = map_hf_to_tone(hf_data)
    return "HuggingFace API", f"لحن (EN): {en} | (FA): {fa}"

In [ ]:
# ===== 7.3 Main Gradio inference function =====

def gradio_predict(user_text, task, model_choice):
    if not user_text or user_text.strip() == "":
        return "لطفاً یک متن وارد کنید."

    # فعلاً فقط Classification (تشخیص لحن) پیاده‌سازی شده
    if task != "Classification (Tone Detection)":
        return "⚠️ فعلاً فقط بخش «تشخیص لحن پیام پشتیبانی» پیاده‌سازی شده است."

    # اگر مدل HuggingFace انتخاب شده:
    if model_choice == "HuggingFace (API)":
        model_name, result = predict_tone_hf(user_text)
        return f"[{model_name}] → {result}"

    # مدل‌های داخلی (SVM, LR, RF, DT)
    model = trained_models.get(model_choice)
    if model is None:
        return "❌ مدل انتخاب‌شده پیدا نشد."

    # استفاده از clean_text + tfidf + مدل انتخاب‌شده
    text_clean = clean_text(user_text)
    vec = tfidf.transform([text_clean])
    pred_id = model.predict(vec)[0]

    label_en = id2label[pred_id]
    label_fa = label2fa[label_en]

    return f"مدل: {model_choice}\nلحن (EN): {label_en}\nلحن (FA): {label_fa}"

In [ ]:
# ===== 7.4 Gradio Blocks UI =====

task_options = [
    "Classification (Tone Detection)",
    "Spam Detection (Coming Soon)",
    "Translation (Coming Soon)"
]

model_options = list(trained_models.keys()) + ["HuggingFace (API)"]

with gr.Blocks(title="Special Topics - NLP Project") as demo:
    # State برای کنترل نمایش صفحه‌ها
    show_main = gr.State(False)

    # ---------- صفحه ۱: Welcome ----------
    with gr.Column(visible=True) as page1:
        gr.Markdown("## درس مباحث ویژه – استاد زاغری")
        gr.Markdown("### پروژه: سیستم تشخیص لحن پیام‌های پشتیبانی مشتری")
        gr.Markdown("**نام دانشجو: فاطمه صديقی‌زاده**")
        gr.Markdown("برای ادامه، روی دکمه زیر کلیک کنید.")
        btn_next = gr.Button("Next ▶️")

    # ---------- صفحه ۲: UI اصلی ----------
    with gr.Column(visible=False) as page2:
        gr.Markdown("## 🧠 Customer Support Tone Checker (فارسی)")
        gr.Markdown("در این بخش می‌توانید متن مشتری را وارد کنید، تسک و مدل را انتخاب کنید و لحن را ببینید.")

        with gr.Row():
            task_dd = gr.Radio(task_options, value="Classification (Tone Detection)", label="Task / نوع کار")
            model_dd = gr.Dropdown(model_options, value="SVM (LinearSVC)", label="Model / نوع مدل")

        # بخش API Key برای HuggingFace
        gr.Markdown("---")
        gr.Markdown("### 🔐 HuggingFace API Key (برای مدل‌های آنلاین – کلید موقت و قابل‌به‌روزرسانی)")
        with gr.Row():
            hf_key_box = gr.Textbox(label="HuggingFace API Key", type="password", placeholder="توکن موقت خود را اینجا وارد کنید...")
            hf_key_btn = gr.Button("Update Key 🔄")
        hf_key_status = gr.Markdown("وضعیت کلید: هنوز تنظیم نشده است.")

        # ورودی متن و خروجی
        gr.Markdown("---")
        user_text = gr.Textbox(lines=4, label="متن پیام مشتری (فارسی)", placeholder="مثال: سلام وقت بخیر، میشه وضعیت سفارشم رو بگید؟")
        run_btn = gr.Button("تشخیص لحن 🚀")
        output_box = gr.Textbox(label="نتیجه", lines=5)

        # رویداد دکمه‌ها
        def go_next():
            return gr.update(visible=False), gr.update(visible=True)

        btn_next.click(fn=go_next, inputs=None, outputs=[page1, page2])

        hf_key_btn.click(
            fn=set_hf_key,
            inputs=hf_key_box,
            outputs=hf_key_status
        )

        run_btn.click(
            fn=gradio_predict,
            inputs=[user_text, task_dd, model_dd],
            outputs=output_box
        )

demo.launch(share=False)